<a href="https://colab.research.google.com/github/karunaprakash062/ml/blob/main/qq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
train_csv=pd.read_csv('/content/drive/MyDrive/train.csv')
cross_validation=pd.read_csv('/content/drive/MyDrive/cleaned_dev_data.csv')
test_cleaned=pd.read_csv('/content/drive/MyDrive/cleaned_test.csv')

In [4]:
# Drop rows that containing missing or null values.
test_cleaned.dropna(inplace=True)
# To reset the index after droping the rows
test_cleaned.reset_index(drop=True,inplace=True)
cross_validation.dropna(inplace=True)
cross_validation.reset_index(drop=True,inplace=True)

In [5]:
# creating a numpy array from the values in text column
text_data=np.array(train_csv['text'])

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re